# Regression Model with Pyspark

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('ecommerce').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [16]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [17]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [19]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [20]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol = 'features')

In [21]:
output = assembler.transform(data)

In [22]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [23]:
final_data = output.select('features', 'Yearly Amount Spent')

In [24]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [25]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [26]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean| 498.10842830169736|
| stddev|  77.44107695816339|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [27]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [28]:
lr_model = lr.fit(train_data)

In [30]:
test_results = lr_model.evaluate(test_data)

In [32]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -11.723227630047063|
| -5.2772831048044395|
|  10.720597822222203|
| -17.196595812837018|
|   4.379703417298174|
|   3.712494851718361|
| -5.5804409274034015|
|  3.5260400457737546|
| -14.209748601461797|
|  17.023105369655525|
|   -4.71434426426714|
|   7.359038055282269|
|  -10.46345308477305|
| -17.219084383006077|
| -2.0471436491972668|
|   8.114229187213482|
|  -17.77209245920085|
|0.052631473207782165|
|   5.397290016253805|
|  0.9950876481177602|
+--------------------+
only showing top 20 rows



In [33]:
test_results.rootMeanSquaredError

9.653502210499852

In [34]:
unlabeled_data = test_data.select('features')

In [35]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.4925366965402...|
|[30.7377203726281...|
|[30.8162006488763...|
|[31.0472221394875...|
|[31.3662121671876...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.5741380228732...|
|[31.6098395733896...|
|[31.7656188210424...|
|[31.8209982016720...|
|[31.8648325480987...|
|[31.9048571310136...|
|[31.9120759292006...|
|[31.9549038566348...|
|[31.9563005605233...|
|[32.0047530203648...|
|[32.0305497162129...|
|[32.0444861274404...|
+--------------------+
only showing top 20 rows



In [36]:
predictions = lr_model.transform(unlabeled_data)

In [37]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|331.65209743324067|
|[30.4925366965402...|287.74852882471896|
|[30.7377203726281...| 451.0601443740077|
|[30.8162006488763...|  283.282936761306|
|[31.0472221394875...| 388.1176957717232|
|[31.3662121671876...|426.87638770476656|
|[31.5257524169682...| 449.5460677372853|
|[31.5316044825729...| 432.9895656835888|
|[31.5741380228732...| 558.6190207620486|
|[31.6098395733896...|427.52244428145264|
|[31.7656188210424...| 501.2684258998743|
|[31.8209982016720...| 417.3162429579311|
|[31.8648325480987...| 450.3547335615867|
|[31.9048571310136...| 491.1689418058222|
|[31.9120759292006...|  389.581859954905|
|[31.9549038566348...| 431.8836507527135|
|[31.9563005605233...| 564.8980242063997|
|[32.0047530203648...|463.69334964742166|
|[32.0305497162129...|  588.877193402358|
|[32.0444861274404...| 447.2347415384322|
+--------------------+------------